### This Notebook contain 2 models .
#### > The first model made through GradientBoost Regressor and cross validation gave the accuracy of 0.044..
#### > The second model through Deep learning didn't givr such accuracy(DK why)

In [19]:
# importing libraries

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from scipy.optimize import minimize

import warnings 
warnings.filterwarnings('ignore')

# Importing Data

In [20]:
# Importing Data
pd.set_option('display.max_columns',None)

# training data
train = pd.read_csv('train.csv')

# test data
test = pd.read_csv('test.csv')
df=pd.concat([train,test], sort=False)
train.head()

,galactic year,galaxy,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",Total unemployment rate (female to male ratio),Vulnerable employment (% of total employment),"Unemployment, total (% of labour force)",Employment in agriculture (% of total employment),Labour force participation rate (% ages 15 and older),"Labour force participation rate (% ages 15 and older), female",Employment in services (% of total employment),"Labour force participation rate (% ages 15 and older), male",Employment to population ratio (% ages 15 and older),Jungle area (% of total land area),"Share of employment in nonagriculture, female (% of total employment in nonagriculture)",Youth unemployment rate (female to male ratio),"Unemployment, youth (% ages 15–24)","Mortality rate, female grown up (per 1,000 people)","Mortality rate, male grown up (per 1,000 people)","Infants lacking immunization, red hot disease (% of one-galactic year-olds)","Infants lacking immunization, Combination Vaccine (% of one-galactic year-olds)",Gross galactic product (GGP) per capita,"Gross galactic product (GGP), total","Outer Galaxies direct investment, net inflows (% of GGP)",Exports and imports (% of GGP),Share of seats in senate (% held by female),Natural resource depletion,"Mean years of education, female (galactic years)","Mean years of education, male (galactic years)","Expected years of education, female (galactic years)","Expected years of education, male (galactic years)","Maternal mortality ratio (deaths per 100,000 live births)",Renewable energy consumption (% of total final energy consumption),"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),y
0,990025,Large Magellanic Cloud (LMC),0.628657,63.125200,27109.234310,0.646039,8.240543,NaN,NaN,NaN,NaN,NaN,NaN,38.305483,681.841086,22.131436,150.259290,129.841789,12.403812,490.446400,60.121580,57.028183,69.825369,46

In [21]:
# Converting Categorical to numeric
df["galaxy"] = df["galaxy"].astype('category')
df["galaxy"] = df["galaxy"].cat.codes

In [22]:
train = df[:3865]
test = df[3865:]
test=test.drop("y", axis = 1)
test_res= test.copy()

## Finding unique Galaxies

In [23]:
# From train set
train_gal=set(train["galaxy"])
s=0
for x in train_gal:
    s=s+len(train.loc[train['galaxy'] == x])
print("Total distinct galaxies: {}".format(len(train_gal)))
print("Average samples per galaxy: {}".format(s/len(train_gal)))

Total distinct galaxies: 181
Average samples per galaxy: 21.353591160220994


In [24]:
# unique galaxies foem test.csv
test_gal=set(test["galaxy"])
s=0
for x in test_gal:
    s=s+len(test.loc[test['galaxy'] == x])
print("Total distinct galaxies: {}".format(len(test_gal)))
print("Average samples per galaxy: {}".format(s/len(test_gal)))

Total distinct galaxies: 172
Average samples per galaxy: 5.174418604651163


In [25]:
print("Train Data shape :", train.shape)
print("Test Data shape :", test.shape)

Train Data shape : (3865, 80)
Test Data shape : (890, 79)


# Cross Validation Training Data

In [26]:
def cross_validation_loop(data,cor):
    labels= data['y']
    data=data.drop('galaxy', axis=1)    
    data=data.drop('y', axis=1)
    
    correlation=abs(data.corrwith(labels))
    columns=correlation.nlargest(cor).index
    data=data[columns]
    
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(data)
    data=imp.transform(data)

    scaler = StandardScaler().fit(data)
    data = scaler.transform(data)
        
    estimator = GradientBoostingRegressor(n_estimators=300)
    
    cv_results = cross_validate(estimator, data, labels, cv=4, scoring='neg_root_mean_squared_error')

    error=np.mean(cv_results['test_score'])
    
    return error

In [32]:
train_gal=set(train["galaxy"])
# train_gal.remove(126)
def loop_train(cor):
    errors=[]
    for gal in train_gal:
        index = train.index[train['galaxy'] == gal]
        data = train.loc[index]
        errors.append(cross_validation_loop(data,cor))
    return np.mean(errors)

In [30]:
cor=[20,10]
errors=[]
for x in cor:
    errors.append(loop_train(x))

In [31]:
print(errors)

[-0.006363456285066111, -0.006499261858746549]


### Corelation of 20 comes out to be best

In [29]:
print(errors)# error of corr 20 and corr 15

[-0.006334945300709283, -0.006384303802044315]


In [ ]:
# (test_res)

In [33]:

from sklearn.metrics import mean_squared_error
from math import sqrt
# Making predictions on Test data
def test_loop(data, test_data):
    labels= data['y']
    data=data.drop('galaxy', axis=1)    
    data=data.drop('y', axis=1)
    correlation=abs(data.corrwith(labels))
    columns=correlation.nlargest(20).index
    
    train_labels = labels
    train_data   = data[columns]
    test_data    = test_data[columns]
    
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(train_data)
    train_data = imp.transform(train_data)
    test_data  = imp.transform(test_data)

    scaler = StandardScaler().fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)

    model = GradientBoostingRegressor(n_estimators=300)
    model.fit(train_data, train_labels)
    
    pred_train = model.predict(train_data)
    print("RMSE on Train Dataset: ",np.sqrt(mean_squared_error(train_labels,pred_train)))

    predictions = model.predict(test_data)
    return (train_data,test_data ,predictions)

In [34]:
test = test_res
test=test.sort_values(by=['galaxy'])
test_pred = pd.DataFrame(0, index=np.arange(len(test)), columns=["predicted_y"])

### Looping over all galaxy types in the test set and making predictions

In [35]:
i=0
for gal in test_gal:
    count=len(test.loc[test['galaxy'] == gal])
    index = train.index[train['galaxy'] == gal]
    data = train.loc[index]
    X_train, X_test,pred=test_loop(data,test.loc[test['galaxy'] == gal])
    test_pred.loc[i:i+count-1,'predicted_y'] = pred
    i=i+count 

RMSE on Train Dataset:  0.0003103933264359414
RMSE on Train Dataset:  0.00030816886438606055
RMSE on Train Dataset:  0.006016337318747309
RMSE on Train Dataset:  0.00030682953030826
RMSE on Train Dataset:  0.00411145755235837
RMSE on Train Dataset:  0.0003103784119077628
RMSE on Train Dataset:  0.00030385422492184314
RMSE on Train Dataset:  0.00031131240582540356
RMSE on Train Dataset:  0.0003136645714162281
RMSE on Train Dataset:  0.0003115709674413767
RMSE on Train Dataset:  0.0003033910768156592
RMSE on Train Dataset:  0.0003046384463673706
RMSE on Train Dataset:  0.00031231771309566466
RMSE on Train Dataset:  0.00030991755054793953
RMSE on Train Dataset:  0.00031020150647779375
RMSE on Train Dataset:  0.00031497839193197607
RMSE on Train Dataset:  0.0003050095481138212
RMSE on Train Dataset:  0.00031451652373319053
RMSE on Train Dataset:  0.00030075119349901876
RMSE on Train Dataset:  0.00030647222955124966
RMSE on Train Dataset:  0.0003110030271692002
RMSE on Train Dataset:  0.000

In [36]:
test["predicted_y"]=test_pred.to_numpy()
test.sort_index(inplace=True)
predictions = test["predicted_y"]

In [37]:
# predictions
X_train.shape

(17, 20)

# Optimization Part

In [38]:
index = predictions
pot_inc = -np.log(index+0.01)+3

In [39]:
p2= pot_inc**2

In [40]:
ss = pd.DataFrame({
    'Index':test.index,
    'pred': predictions,
    'opt_pred':0,
    'eei':test['existence expectancy index'], # So we can split into low and high EEI galaxies
})

In [41]:
ss.loc[p2.nlargest(400).index, 'opt_pred']=100
ss=ss.sort_values('pred')
ss.iloc[400:600].opt_pred = 50
ss=ss.sort_index()

In [42]:
increase = (ss['opt_pred']*p2)/1000

In [43]:
print(sum(increase), ss.loc[ss.eei < 0.7, 'opt_pred'].sum(), ss['opt_pred'].sum())

1790.6108407399379 6500 50000


In [44]:
ss[['Index', 'pred', 'opt_pred']].to_csv('submission.csv', index=False)

# Using Deep Learning Model

In [ ]:
train.shape

In [ ]:
test_res.shape

In [ ]:
def preprocessing_train_data(data):
    labels= data['y']
    data=data.drop('galaxy', axis=1)    
    data=data.drop('y', axis=1)
    
    # Removing missing values
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(data)
    data=imp.transform(data)
    
    # Standardizing Values
    scaler = StandardScaler().fit(data)
    data = scaler.transform(data)
    
    return (data,labels)

In [ ]:
train_X, train_Y =  preprocessing_train_data(train)
 

In [ ]:
def preprocessing_test_data(data):
    data=data.drop('galaxy', axis=1)    
    
    # Removing missing values
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(data)
    data=imp.transform(data)
    
    # Standardizing Values
    scaler = StandardScaler().fit(data)
    data = scaler.transform(data)
    
    return data

In [ ]:
X_test = preprocessing_test_data(test_res)

In [ ]:
len(X_test)

In [ ]:
# importing Libraries
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
# Model Architecture
def build_model():
    model = Sequential()

    # Input Layer
    model.add(Dense(128,kernel_initializer='normal', input_dim= train_X.shape[1], activation = 'relu' ))
    
    #Hidden Layers
    model.add(Dense(256, kernel_initializer='normal',activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, kernel_initializer='normal',activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, kernel_initializer='normal',activation='relu'))
    

    # Output Layer
    model.add(Dense(1, kernel_initializer='normal',activation='linear'))
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mean_absolute_error', optimizer = optimizer, metrics=['mean_absolute_error', 'mse'])
    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
# plotting libraries
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [ ]:
EPOCHS = 500

history = model.fit(
  train_X, train_Y,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,)
#   callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
history

In [ ]:
# hist = history.history
mae = history.history['mean_absolute_error']
val_mae = history.history['val_mean_absolute_error']
epochs = range(1, len(mae) + 1)
plt.plot(epochs, mae, 'bo', label='Training MAE')
plt.plot(epochs, val_mae, 'b', label='Validation MAE')
plt.title('Training and validation MAE')
plt.legend()
plt.figure()
plt.show()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [ ]:
plotter.plot({'Basic': history}, metric = "mean_absolute_error")
plt.ylim([0, 0.05])
plt.ylabel('MAE [MPG]')

In [ ]:
# loss, mae, mse = model.evaluate(X_test, test_labels, verbose=2)

# print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

In [ ]:
X_test.shape

In [ ]:
predicted_val_from_tf = model.predict(X_test)

In [ ]:
predicted_val_from_tf

In [ ]:
type(predicted_val_from_tf)

In [ ]:
# train.index[train['galaxy'] == gal]

In [ ]:
X_test

In [ ]:
test['existence expectancy index']

In [ ]:
prediction = pd.DataFrame(predicted_val_from_tf,)

In [ ]:
new_prediction = predicted_val_from_tf.astype('float')

In [ ]:
predict = pd.Series(new_prediction.flatten())

In [ ]:
predict

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
pred_train = model.predict(train_features)
print("RMSE on Train Dataset: ",np.sqrt(mean_squared_error(train_labels,pred_train)))


## Making Submission File

In [ ]:
# def make_submission(predictions):
pot_inc = -np.log(index+0.01)+3
p2= pot_inc**2
my_submission = pd.DataFrame({
'Index':test.index,
'pred': predict,
'opt_pred':0,
'eei':test['existence expectancy index'],})

my_submission.loc[p2.nlargest(400).index, 'opt_pred']=100
my_submission=my_submission.sort_values('pred')
my_submission.iloc[400:600].opt_pred = 50
my_submission=my_submission.sort_index()

increase = (ss['opt_pred']*p2)/1000

#     print(sum(increase), my_submission.loc[my_submission.eei < 0.7, 'opt_pred'].sum(), my_submission['opt_pred'].sum())
    
#     return my_submission
# #     my_submission[['Index', 'pred', 'opt_pred']].to_csv('submission.csv', index=False)


In [ ]:
# my_submission[['Index', 'pred', 'opt_pred']].to_csv('submission.csv', index=False)

In [ ]:
# making cleaned trained and test data

def cleanend_train_csv(train_data):
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(train_data)
    train_data=imp.transform(train_data)
    X_imputed_df = pd.DataFrame(train_data, columns = train.columns)
    return X_imputed_df
    
    

In [ ]:
clean_train_data = cleanend_train_csv(train)

In [ ]:
# clean_train_data.to_csv('Cleaned_train.csv')

In [ ]:
def cleanend_test_csv(test_data):
    imp = SimpleImputer(missing_values=np.nan, strategy='mean').fit(test_data)
    test_data=imp.transform(test_data)
    X_imputed_df = pd.DataFrame(test_data, columns = test.columns)
    return X_imputed_df

In [ ]:
clean_test_data = cleanend_test_csv(test)

In [ ]:
# clean_test_data.to_csv('Cleaned_test.csv')